# Clase: Similitud Semántica y Sistemas de Anotación NER

Esta notebook cubre los siguientes puntos:

**5.c.i)** Armado de un _dataset_ con oraciones.  
**5.c.ii)** Métodos de medición de similitud semántica:  

&nbsp;&nbsp;• Similitud coseno (TF‑IDF)  
&nbsp;&nbsp;• Similitud por presencia de tokens (Jaccard)  
&nbsp;&nbsp;• Word Emebddings  
&nbsp;&nbsp;    • Word2Vec (vectores de palabras)  
&nbsp;&nbsp;    • FastText (sub‑word embeddings)  

**5.b.i)** Sistemas de anotación: BIO / BILOU (con mención a BOU).  
**5.b.ii)** NER con **spaCy** y **Stanza**, y una demo básica de extracción de relaciones.

---


In [ ]:
import pandas as pd


sentences = [
"La inteligencia artificial está transformando la industria del software.",
"La IA revolucionará la asistencia médica en los próximos años.",
"Los avances en inteligencia artificial y aprendizaje automático impulsan nuevas aplicaciones.",
"Los goles de Lionel Messi llevaron al equipo a la victoria.",
"Las estrategias defensivas del fútbol moderno requieren comunicación constante.",
"El equipo de fútbol ganó el campeonato después de un partido intenso."
]
df = pd.DataFrame(sentences, columns=["text"])


## 2. Métodos de medición de similitud semántica (5.c.ii)

Veremos cuatro enfoques:

1. **Similitud coseno** entre vectores TF‑IDF.  
2. **Similitud Jaccard** basada en presencia/ausencia de tokens (bolsa de palabras binaria).  
3. **Word2Vec** – calculando la similitud promedio de vectores de palabras (usaremos `es_core_news_md`).  
4. **FastText** – similar al anterior, aprovechando sub‑palabras (opcional si descargas un modelo).

Cada bloque compara la primera oración con el resto, a modo de ejemplo.


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

stopwords = ['de', 'la', 'y', 'en', 'a', 'el', 'que', 'los', 'del', 'con', 'por', 'un', 'una', 'me', 'fue', 'tan', 'muy', 'este', 'es', 'para', 'se', 'lo', 'como', 'al', 'si', 'con', 'su', 'misma']

tfidf = TfidfVectorizer(stop_words=stopwords)
X = tfidf.fit_transform(df['text'])

# Similitud coseno respecto a la primera oración
cosine_scores = cosine_similarity(X[0], X).flatten()
pd.DataFrame({'oracion': df['text'], 'sim_coseno': cosine_scores}).sort_values('sim_coseno', ascending=False).head(5)


,oracion,sim_coseno
0,La inteligencia artificial está transformando ...,1.000000
1,La inteligencia artificial revolucionará la as...,0.176353
2,Los avances en inteligencia artificial y apren...,0.163190
3,Los goles de Lionel Messi llevaron al equipo a...,0.000000
4,Las estrategias defensivas del fútbol moderno ...,0.000000


### Indice Jaccard

La **similitud de Jaccard** entre dos conjuntos \(A\) y \(B\) se define como

$$
\text{Jaccard}(A,B) = \frac{|A \cap B|}{|A \cup B|}
$$

ddonde  

* $\displaystyle \left| A \cap B \right|$ — **número de elementos comunes** (intersección)  
* $\displaystyle \left| A \cup B \right|$ — **número total de elementos únicos** presentes en al menos uno de los conjuntos (unión)

### Ejemplo paso a paso

Supongamos dos oraciones tokenizadas (sin *stop-words*):

| Oración | Tokens |
|---------|--------|
| **A**: “La inteligencia artificial avanza rápido” | { inteligencia, artificial, avanza, rápido } |
| **B**: “Los avances en inteligencia artificial son impresionantes” | { avances, inteligencia, artificial, impresionantes } |

---
**Intersección**

$$
A \cap B \;=\; \{\textit{inteligencia},\ \textit{artificial}\}
$$

$$
\bigl|A \cap B\bigr| \;=\; 2
$$
**Unión**

$$
A \cup B = \{\textit{inteligencia},\ \textit{artificial},\ \textit{avanza},\ \textit{rápido},\ \textit{avances},\ \textit{impresionantes}\}
$$

$$
\bigl|A \cup B\bigr| = 6
$$

**Cálculo**

$$
J(A,B)= \frac{\bigl|A \cap B\bigr|}{\bigl|A \cup B\bigr|}= \frac{2}{6}\approx 0.33
$$

La similitud de Jaccard indica que las frases comparten aproximadamente un **tercio** de su vocabulario “informativo”.

In [18]:
import numpy as np

def jaccard_similarity(a, b):
    set_a, set_b = set(a.split()), set(b.split())
    inter = set_a & set_b
    union = set_a | set_b
    return len(inter) / len(union)

jaccard_scores = df['text'].apply(lambda x: jaccard_similarity(df['text'][0], x))
pd.DataFrame({'oracion': df['text'], 'sim_jaccard': jaccard_scores}).sort_values('sim_jaccard', ascending=False).head(6)


,oracion,sim_jaccard
0,La inteligencia artificial está transformando ...,1.000000
1,La inteligencia artificial revolucionará la as...,0.250000
2,Los avances en inteligencia artificial y apren...,0.111111
4,Las estrategias defensivas del fútbol moderno ...,0.058824
3,Los goles de Lionel Messi llevaron al equipo a...,0.052632
5,El equipo de fútbol ganó el campeonato después...,0.000000


In [21]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 18.5 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [24]:
import spacy, pathlib, subprocess, sys

# Asegurate de tener instalado el modelo con: python -m spacy download es_core_news_md

nlp = spacy.load('es_core_news_md')

def doc_vector(text):
    doc = nlp(text)
    return doc.vector


vecs = df['text'].apply(doc_vector).tolist()
matrix = np.vstack(vecs)
w2v_scores = cosine_similarity([matrix[0]], matrix).flatten()
pd.DataFrame({'oracion': df['text'], 'sim_word2vec': w2v_scores}).sort_values('sim_word2vec', ascending=False).head(6)


,oracion,sim_word2vec
0,La inteligencia artificial está transformando ...,1.000000
1,La inteligencia artificial revolucionará la as...,0.845334
2,Los avances en inteligencia artificial y apren...,0.505219
4,Las estrategias defensivas del fútbol moderno ...,0.483296
3,Los goles de Lionel Messi llevaron al equipo a...,0.457592
5,El equipo de fútbol ganó el campeonato después...,0.414953


In [ ]:
# ---- FastText (opcional) ----
# Este bloque descarga un modelo FastText de 1M dimensiones. Puede tardar y consumir RAM.
# from gensim.models.fasttext import load_facebook_model
# model = load_facebook_model('cc.es.300.bin')
# def ft_sentence_vector(text):
#     tokens = text.split()
#     vecs = [model.wv[word] for word in tokens if word in model.wv]
#     return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)
# ft_vecs = df['texto'].apply(ft_sentence_vector).tolist()
# ft_matrix = np.vstack(ft_vecs)
# ft_scores = cosine_similarity([ft_matrix[0]], ft_matrix).flatten()
# pd.DataFrame({'oracion': df['texto'], 'sim_fasttext': ft_scores}).sort_values('sim_fasttext', ascending=False).head(6)
# print("Consulta comentada: descomenta si tienes un modelo FastText descargado.")


## Entities y NER

### Entidades

## 3. Sistemas de anotación BIO / BILOU (5.b.i)

En **NLP** se usan esquemas que indican qué tokens pertenecen a entidades nombradas.

| Esquema | Descripción | Ejemplo (`ORG` = *Apple*) |
|---------|-------------|---------------------------|
| **BIO** | **B**egin, **I**nside, **O**utside | Apple = **B-ORG**, Inc. = **I-ORG** |
| **BILOU** | **B**egin, **I**nside, **L**ast, **O**utside, **U**nit (entidad de un solo token) | Apple (**U-ORG**) / University of California = B-ORG I-ORG L-ORG |
| **BOU** | Variante simplificada: **B**egin, **O**utside, **U**nit | (poco usada hoy, incluida por completitud) | Apple (**U-ORG**) |

A continuación generamos etiquetas BIO y BILOU para nuestras oraciones usando *spaCy*.


In [ ]:
from spacy.training import offsets_to_biluo_tags


texto = "Apple Inc. lanzó el nuevo iPhone en California."
doc = nlp(texto)

# Obtenén spans de entidades [(start_char, end_char, label), ...]
ents_offsets = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
biluo_tags = offsets_to_biluo_tags(doc, ents_offsets)
bio_tags = [tag.replace('L-', 'I-').replace('U-', 'B-') for tag in biluo_tags]

for token, biluo, bio in zip(doc, biluo_tags, bio_tags):
    print(f"{token.text:<12} {biluo:<8} {bio}")


Apple        B-ORG    B-ORG
Inc          L-ORG    I-ORG
.            O        O
lanzó        O        O
el           O        O
nuevo        O        O
iPhone       U-MISC   B-MISC
en           O        O
California   U-LOC    B-LOC
.            O        O


## 4. NER con spaCy + Extracción de relaciones (5.b.ii)

A continuación comparamos los *pipelines* de reconocimiento de entidades.


In [ ]:

doc = nlp("Barcelona FC fichó a Lionel Messi en 2003.")
print("spaCy:", [(ent.text, ent.label_) for ent in doc.ents])

def extract_svo(doc):
    svos = []
    for token in doc:
        if token.dep_ == "ROOT":
            subj = [w for w in token.lefts if w.dep_.startswith("nsubj")]
            obj  = [w for w in token.rights if w.dep_.startswith(("dobj", "obj"))]
            if subj and obj:
                svos.append((subj[0].text, token.text, obj[0].text))
    return svos


print("SVO:", extract_svo(doc))


spaCy: [('Barcelona FC', 'ORG'), ('Lionel Messi', 'PER')]
SVO: [('Barcelona', 'fichó', 'Lionel')]


In [34]:

def _span_for_ent_token(token):
    """Devuelve el texto completo de la entidad que contiene al token
       o None si el token no pertenece a ninguna entidad."""
    if token.ent_type_ == "":
        return None
    for ent in token.doc.ents:
        if ent.start <= token.i < ent.end:
            return ent.text
    return None

def _expand_to_np(token):
    """Devuelve el texto del sintagma nominal gobernado por el token."""
    subtree = list(token.subtree)
    left   = subtree[0].i
    right  = subtree[-1].i + 1
    return token.doc[left:right].text

def extract_svo_ent(doc):
    svos = []
    for root in doc:
        if root.dep_ == "ROOT":
            subjs = [w for w in root.lefts  if w.dep_.startswith("nsubj")]
            objs  = [w for w in root.rights if w.dep_.startswith(("dobj", "obj"))]
            if subjs and objs:
                def full_phrase(tok):
                    # 1) usa la entidad completa si existe
                    span = _span_for_ent_token(tok)
                    if span:
                        return span
                    # 2) si no, usa el sintagma nominal
                    return _expand_to_np(tok)
                svos.append((full_phrase(subjs[0]),
                              root.lemma_,                # verbo en lema
                              full_phrase(objs[0])))
    return svos



In [35]:
print("spaCy:", [(ent.text, ent.label_) for ent in doc.ents])
print("SVO:", extract_svo_ent(doc))

spaCy: [('Barcelona FC', 'ORG'), ('Lionel Messi', 'PER')]
SVO: [('Barcelona FC', 'fichar', 'Lionel Messi')]


---

### Conclusiones

* Hemos creado un dataset pequeño y probado varias métricas de similitud.  
* Vimos cómo funcionan los esquemas de anotación y aplicamos **spaCy** y **Stanza** para NER.  
* La extracción de relaciones se ilustró con un sencillo *pattern S‑V‑O*.  

**Próximos pasos sugeridos**  

1. Ampliar el dataset y comparar correlaciones entre las métricas.  
2. Entrenar un clasificador con embeddings promedio y probar su desempeño.  
3. Probar `spacy-extractruler` o librerías específicas de RE (Relation Extraction) como `spacy-relation-extractor`.  

¡Éxitos en tu estudio!